In [1]:
#for lmax=8
import numpy as np
from scipy import linalg
#from itertools import permutations
from itertools import combinations
#from itertools import combinations_with_replacement
from scipy.optimize import minimize
from scipy.optimize import check_grad
from scipy.special import binom
from numpy.linalg import multi_dot
from timeit import default_timer as timer
import pandas as pd
import csv

In [2]:
# set some constants
N = 40
c=0.0
g_4=0
g_3=0.0
omega_length = 8                # lmax
max_length = 2*omega_length-2   # The largest loop generated has length 2*omega_length - 2
start=0
end=0

# New Section

In [3]:
# creat empty list
loop_list =[]                     # loop_list is a list of arrays, as arrays (loops) have different lengths
loop_list.append(np.array([]))
numb_tuples_list = []
numb_tuples_list.append((start, end))

In [4]:
def compare (Loop1, Loop2):

  """
  Compare two loops up to cyclic permutations
  """
  Length1 = len (Loop1)
  Length2 = len (Loop2)

  Same=False
  if (Length2 == 0) :
          Same=True
          return Same

  #Create warning if loops passed with different lengths #print ("lengths", Length1, Length2)

  Same = False
  for i in range (Length2):
    if np.array_equal (Loop1, np.roll(Loop2, i)):
        Same=True
        break

  return Same

In [5]:
def cyclic_generator (lp):

    """

    Input:   lp is a (1,len(lp)) ndarray
    Output:  is an array containing the first of the lexicographically sorted set of cyclic permutations of lp
             - example: [2,1,2,1] --> [1,2,1,2]
    """


    cyclic_perm = np.array([[0]*len(lp)]*len(lp))

    for i in range (len(lp)):
      cyclic_perm[i]= np.roll(lp, i)

    first_lexi_array = np.array (sorted([tuple(row) for row in cyclic_perm]))[0:1, :]

    return first_lexi_array

In [6]:
def loops_of_length (loop_length):

    """
    Generates all distinct loops (up to cyclic permutations) of length loop_length
    Returns: Stack (matrix) of (1,loop_length) loop arrays

    """

    print("starting loops of length ", loop_length)

    """ Old code ---- uses set to remove identical permutations --- slow for large loops

    cyclic_set = cyclic_generator (np.array ([2]*loop_length)) #need for cyclic generator?
    big_big_perm_array=cyclic_set

    for i in range (1,loop_length):

        print ("starting with", i, "[1]s" )
        start_list = [1]*(i) + [2]*(loop_length-i)
        perm = permutations (start_list, loop_length) # list of permutations
        perm = np.array (sorted (set(perm)))          # removes identical permutations (set has only distinct elements)
        print ("sorted set generated")                                              # and converts to array
        big_perm_array = np.array([[0]*(loop_length)]*len(perm))
        for p in range (len(perm)):
            cyclic_set = cyclic_generator (perm [p])
            big_perm_array[p] = cyclic_set            # stores first of the lexicographically sorted cyclic permutations
        print ("removed identical cyclic permutations")                                              # in big_perm_array
        big_big_perm_array = np.vstack((big_big_perm_array,big_perm_array))

    cyclic_set = cyclic_generator (np.array ([1]*loop_length))
    big_big_perm_array = np.vstack((big_big_perm_array,cyclic_set))

    big_big_perm_set = np.array (sorted({tuple(row) for row in big_big_perm_array}))
    """
    """ Version -- 1  """

    all_ones = np.array ([1]*loop_length)
    big_big_perm_array=all_ones

    for i in range (1,loop_length):

        print ("starting with", i, "[2]s" )
        #no_2s=i
        comb = list(combinations (range(loop_length),i))
        #print ("print combinations")
        #for x in comb:
        #    print (x)
        #no_combinations = combinations (range(loop_length),i)
        #start=timer()
        #no_combinations=binom(loop_length,i)
        #end=timer()
        #print ("no_combinations", no_combinations, "time taken" , end-start)

        #start=timer()
        numb_combinations=len(comb)
        #end=timer()
        #print ("no_combinations with length", no_combinations, "time taken" , end-start)


        #big_perm_array = np.array([[0]*(loop_length)]*len(comb)) #may be avoidable by appending
        big_perm_array = np.array([[0]*(loop_length)]*numb_combinations)
        for p in range(numb_combinations):
            perm_distinct=list (all_ones)
            #print ("perm_distinct initial ")
            #print (perm_distinct)

            for comb_index in comb[p]:
                perm_distinct[comb_index]= 2
            #print ("perm_distinct for p ", p)
            #print (perm_distinct)
            big_perm_array[p]=cyclic_generator(np.array(perm_distinct))
        #print ("big_perm_array generated")
        #print (big_perm_array)                                             # and converts to array
        big_perm_array = np.array (sorted({tuple(row) for row in big_perm_array}))
        #print ("big_perm_array set generated")
        #print (big_perm_array)
        big_big_perm_array = np.vstack((big_big_perm_array,big_perm_array))

    big_big_perm_array = np.vstack((big_big_perm_array,np.array ([2]*loop_length)))

    #big_big_perm_set = np.array (sorted({tuple(row) for row in big_big_perm_array}))

    """
    Version 2 -- trying to iterate directly on combinations --- challenges with lists and arrays

    all_ones = np.array ([1]*loop_length)
    #cyclic_set = cyclic_generator (np.array ([2]*loop_length)) #need for cyclic generator?
    big_big_perm_array=all_ones

    for i in range (1,loop_length):

        print ("starting with", i, "[2]s" )
        #no_2s=i
        #comb = list(combinations (range(loop_length),i))
        #print ("print combinations")
        #for x in comb:
        #    print (x)
        #no_cominations = combinations (range(loop_length),i)
        #big_perm_array = np.array([[0]*(loop_length)]*len(comb)) #may be avoidable by appending
        big_perm_array=[]
        print ("big_perm_array empty")
        print (big_perm_array)
        print ("type", type(big_perm_array))
        for comb_tuple in combinations (range(loop_length),i):
            perm_distinct=list (all_ones)
            print ("perm_distinct initial ")
            print (perm_distinct)

            for comb_index in comb_tuple:
                perm_distinct[comb_index]= 2
            print ("perm_distinct for comb_tuple ", comb_tuple)
            print (perm_distinct)
            big_perm_array.append(cyclic_generator(np.array(perm_distinct)))
            #big_perm_array.append(cyclic_generator((perm_distinct)))
        print ("big_perm_array generated")
        print (big_perm_array)
        print ("type", type(big_perm_array))                                          # and converts to array
        big_perm_array = np.array (sorted({tuple(row) for row in np.array(big_perm_array)}))
        print ("big_perm_array set generated")
        print (big_perm_array)
        big_big_perm_array = np.vstack((big_big_perm_array,big_perm_array))

    big_big_perm_array = np.vstack((big_big_perm_array,np.array ([2]*loop_length)))

    """


    return big_big_perm_array


In [7]:
def effect_pot(x_N_2, omega_dim, num_loops,litte_omega_dim, nt_hooft, c_coupling, g3_coupling,g4_coupling):

    global loop
    global omega
    global little_omega
    global LnJ
    """ imported - checked
    matrices_array =np.zeros((2,nt_hooft,nt_hooft),dtype=complex)
    matrices_array[0] = np.diag(x_N_2[0:nt_hooft])
    #print ("full matrix 1", matrices_array[0]) #keep it real for the time being
    temp_matrix_2 = np.reshape (x_N_2[nt_hooft:nt_hooft**2+nt_hooft], (nt_hooft,nt_hooft))
    real_matrix_2 = np.triu(temp_matrix_2)
    real_matrix_2 = real_matrix_2+real_matrix_2.T-np.diag(np.diag(temp_matrix_2))
    imag_matrix_2 = np.tril(temp_matrix_2)
    imag_matrix_2 = imag_matrix_2-imag_matrix_2.T
    matrices_array[1]=real_matrix_2+imag_matrix_2*1.j
    verified same as v1 """
    ##print ("Called function")
    #  Extract info from flat array
    #print ("Started function")#
    #print ("N =", nt_hooft, "length = ", len(x_N_2))#
    #matrix_1_diagonal= x_N_2[0:nt_hooft]
    

    # Create 2 N*N arrays wit =h complex entries
    matrices_array =np.zeros((2,nt_hooft,nt_hooft),dtype=complex)


    # First array is a diagonal matrix with the first N elements of input x_N_2
    matrices_array[0] = np.diag(x_N_2[0:nt_hooft])

    #second array is full N*N matrix with the remaining N**2 elements of input x_N_2. 
    temp_matrix_2 = np.reshape (x_N_2[nt_hooft:nt_hooft**2+nt_hooft], (nt_hooft,nt_hooft))
    
    real_matrix_2 = np.triu(temp_matrix_2)
    real_matrix_2 = real_matrix_2+real_matrix_2.T-np.diag(np.diag(temp_matrix_2))
    imag_matrix_2 = np.tril(temp_matrix_2)
    imag_matrix_2 = imag_matrix_2-imag_matrix_2.T
    #print ("real part", real_matrix_2)
    #print ("imaginary part", imag_matrix_2)
    matrices_array[1]=real_matrix_2+imag_matrix_2*1.j
    ##print ("full matrix 2", matrices_array[1])
    #print (np.dot (matrix_1,matrix_2))
    #matrices_array =
    #print ("array of matrices", matrices_array)

    """
    for i in range (1,num_loops+1):
        loop_matrix=matrices_array[loop_list[i][0]-1]
        #print (loop_list[i], loop_list[i][0]-1, len(loop_list[i]))
        #print (loop_matrix)
        for k in range(1,len(loop_list[i])):
            #print (k)
            #loop_matrix=loop_matrix @ matrices_array[loop_list[i][k]-1]
            loop_matrix=np.dot(loop_matrix,matrices_array[loop_list[i][k]-1])
        #print("after k loop, loop matrix", loop_matrix)
        #print ("Are loops real ? ", np.trace(loop_matrix) /( np.power(nt_hooft, (len(loop_list[i])+2)/2)))
        loop[i]=np.trace(loop_matrix) /( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #note: not declared as complex; always returns real value
        #print ("loop[",i,"]", loop[i])
    loop[0]=1.0

    """
    for i in range (1,num_loops+1):
        loop_matrix=matrices_array[loop_list[i][0]-1]
        #print (loop_list[i], loop_list[i][0]-1, len(loop_list[i]))
        #print (loop_matrix)
        for k in range(1,len(loop_list[i])):
            #print ("loop", i, "word", k, "loop_list", loop_list[i][k], (loop_list[i][k]==1))
            if loop_list[i][k]==1 :
                loop_matrix=loop_matrix * x_N_2[0:nt_hooft]          #[k]#matrices_list[loop_list[i][k]-1]
            else:
                loop_matrix=np.dot(loop_matrix,matrices_array[1])#[loop_list[i][k]-1]
        #print("after k loop, loop matrix", loop_matrix)
        #loop[i]=(np.trace(loop_matrix)).real /( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #v3
        #print ("loop[",i,"]", loop[i])
        loop[i]=np.trace(loop_matrix)/( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #v3
        #print ("loop[",i,"]", loop[i])
    loop[0]=1.0


    """ old code
    ##for i in range (num_loops+1):
        ##print ("loop[",i,"]", loop[i])
    #start building big omega
    #omega = [[(i+1)*(j+1)*loop[i+j]for i in range(omega_size)]for j in range(omega_size)]

    for i in range (omega_dim):
        for j in range (omega_dim):
            omega[i,j]=0.
            for k in range (num_loops+1):
                omega[i,j]+= y[i+1,j+1,k] * loop[k]

    ##print ("omega", omega)

    #print ("eigenvalues", linalg.eigvals(omega))


    for i in range (2,omega_dim):
        little_omega[i]=0.
        for j in range (litte_omega_dim+1):
            for k in range (litte_omega_dim+1):
                little_omega[i]+=z[i+1,j,k]*loop[j]*loop[k]

    ##print ("little_omega", little_omega)
    LnJ = np.linalg.solve(omega, little_omega)

    """
    """ np.sum is faster but memory demanding
    omega=np.sum (np.array([[[y[i+1,j+1,k] * loop[k] for i in range (omega_dim)] for j in range (omega_dim)] for k in range (num_loops+1)]),axis=0)

    little_omega=np.sum (np.array([[[z[i+1,j,k]*loop[j]*loop[k] for i in range (omega_dim)] for j in range (litte_omega_dim+1)] for k in range (litte_omega_dim+1)]),axis=(0,1))
    #little_omega+=- omega[:,2]/2.0-omega[:,4]/2.0 + c_coupling*omega[:,3]-g3_coupling*omega[:,5]/3.0-g3_coupling*omega[:,8]/3.0
    little_omega+=- omega[:,2]/2.0-omega[:,4]/2.0 + c_coupling*omega[:,3]-g3_coupling*omega[:,5]/3.0-g3_coupling*omega[:,8]/3.0-g4_coupling*omega[:,9]-g4_coupling*omega[:,14]
    """

    omega=np.zeros ((omega_dim,omega_dim),dtype=complex)
    for i in range (omega_dim):
        for j in range (omega_dim):
            omega[i,j]=0.0
            for k in range (num_loops+1):
                omega[i,j]+=y[i+1,j+1,k] * loop[k]

    little_omega=np.zeros ((omega_dim),dtype=complex)
    for i in range (omega_dim):
        little_omega[i]=0.0
        for j in range (litte_omega_dim+1):
            for k in range (j,litte_omega_dim+1):
                little_omega[i]+= z[i+1,j,k]*loop[j]*loop[k]
    #little_omega+=- omega[:,2]/2.0-omega[:,4]/2.0 + c_coupling*omega[:,3]-g3_coupling*omega[:,5]/3.0-g3_coupling*omega[:,8]/3.0-g4_coupling*omega[:,9]-g4_coupling*omega[:,14]





    """
    det=np.linalg.det(omega)
    c=np.array(omega)
    #print (c)
    for j in range(omega_dim):
        c[:,j]=little_omega
        LnJ[j]=np.linalg.det(c)/det
        c[:,j]=omega[:,j]
    """

    LnJ=np.linalg.solve(omega,little_omega)

    # Obtain value of collective effective potential
    large_n_energy = np.dot(little_omega , LnJ)/8.0 + (1/2)*loop[3] + (1/2)*loop[5] + g4_coupling*loop[10]+ g4_coupling*loop[15] 

    print("large_n_energy", large_n_energy)


    return large_n_energy.real

In [8]:
def effect_pot_grad(x_N_2, omega_dim, num_loops,litte_omega_dim, nt_hooft, c_coupling, g3_coupling, g4_coupling):

    #global loop
    #global omega
    #global little_omega
    ##print ("Called gradient")
    #  Extract info from flat array
    #print ("Started function")#
    #print ("N =", nt_hooft, "length = ", len(x_N_2))#
    #matrix_1_diagonal= x_N_2[0:nt_hooft]
    matrices_array =np.zeros((2,nt_hooft,nt_hooft),dtype=complex)
    #print (matrices_array)#
    matrices_array[0] = np.diag(x_N_2[0:nt_hooft])
    #matrices_array[0]=matrix_1
    ##print ("full matrix 1", matrices_array[0])
    temp_matrix_2 = np.reshape (x_N_2[nt_hooft:nt_hooft**2+nt_hooft], (nt_hooft,nt_hooft))
    real_matrix_2 = np.triu(temp_matrix_2)
    real_matrix_2 = real_matrix_2+real_matrix_2.T-np.diag(np.diag(temp_matrix_2))
    imag_matrix_2 = np.tril(temp_matrix_2)
    imag_matrix_2 = imag_matrix_2-imag_matrix_2.T
    #print ("real part", real_matrix_2)
    #print ("imaginary part", imag_matrix_2)
    matrices_array[1]=real_matrix_2+imag_matrix_2*1.j
    ##print ("full matrix 2", matrices_array[1])
    #print (np.dot (matrix_1,matrix_2))
    #matrices_array =
    #print ("array of matrices", matrices_array)
    """ older version
    for i in range (1,num_loops+1):
        loop_matrix=matrices_array[loop_list[i][0]-1]
        #print (loop_list[i], loop_list[i][0]-1, len(loop_list[i]))
        #print (loop_matrix)
        for k in range(1,len(loop_list[i])):
            #print (k)
            #loop_matrix=loop_matrix @ matrices_array[loop_list[i][k]-1]
            loop_matrix=np.dot(loop_matrix,matrices_array[loop_list[i][k]-1])
        #print("after k loop, loop matrix", loop_matrix)
        #up to lmax=4 all loops real_matrix_2
        #print ("Are loops real ? ", np.trace(loop_matrix) /( np.power(nt_hooft, (len(loop_list[i])+2)/2)))
        loop[i]=np.trace(loop_matrix) /( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #note: not declared as complex; always returns real value
        #print ("loop[",i,"]", loop[i])
    loop[0]=1.0

    """
    for i in range (1,num_loops+1):
        loop_matrix=matrices_array[loop_list[i][0]-1] #add first matrix of loop[i] to loop matrix
        #print (loop_list[i], loop_list[i][0]-1, len(loop_list[i]))
        #print (loop_matrix)
        for k in range(1,len(loop_list[i])):
            #print ("loop", i, "word", k, "loop_list", loop_list[i][k], (loop_list[i][k]==1))
            if loop_list[i][k]==1 :
                loop_matrix=loop_matrix * x_N_2[0:nt_hooft]          #[k]#matrices_list[loop_list[i][k]-1]
            else:
                loop_matrix=np.dot(loop_matrix,matrices_array[1])#[loop_list[i][k]-1]
        #print("after k loop, loop matrix", loop_matrix)
        #loop[i]=(np.trace(loop_matrix)).real /( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #v3
        loop[i]=np.trace(loop_matrix)/( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #v3

        #print ("loop[",i,"]", loop[i])
    loop[0]=1.0

    #print ("geenrated loops")

    ##for i in range (num_loops+1):
        ##print ("loop[",i,"]", loop[i])

    #start building big omega
    #omega = [[(i+1)*(j+1)*loop[i+j]for i in range(omega_size)]for j in range(omega_size)]
    """ Old code
    for i in range (omega_dim):
        for j in range (omega_dim):
            omega[i,j]=0.
            for k in range (num_loops+1):
                omega[i,j]+= y[i+1,j+1,k] * loop[k]

    ##print ("omega", omega)

    #print ("eigenvalues", linalg.eigvals(omega))
    #inconsistency with omega_size=omega_dim
    for i in range (2,omega_dim):
        little_omega[i]=0.
        for j in range (litte_omega_dim+1):
            for k in range (litte_omega_dim+1):
                little_omega[i]+=z[i+1,j,k]*loop[j]*loop[k]

    ##print ("little_omega", little_omega)

    LnJ = np.linalg.solve(omega, little_omega)
    """
    """
    #print ("starting Omega")
    omega=np.sum (np.array([[[y[i+1,j+1,k] * loop[k] for i in range (omega_dim)] for j in range (omega_dim)] for k in range (num_loops+1)]),axis=0)
    #print ("starting little omega")
    little_omega=np.sum (np.array([[[z[i+1,j,k]*loop[j]*loop[k] for i in range (omega_dim)] for j in range (litte_omega_dim+1)] for k in range (litte_omega_dim+1)]),axis=(0,1))
    #little_omega+=- omega[:,2]/2.0-omega[:,4]/2.0+ c_coupling*omega[:,3]-g3_coupling*omega[:,5]/3.0-g3_coupling*omega[:,8]/3.0
    little_omega+=- omega[:,2]/2.0-omega[:,4]/2.0+ c_coupling*omega[:,3]-g3_coupling*omega[:,5]/3.0-g3_coupling*omega[:,8]/3.0-g4_coupling*omega[:,9]-g4_coupling*omega[:,14]
    """
    #print ("started inversion")

    omega=np.zeros ((omega_dim,omega_dim),dtype=complex)
    for i in range (omega_dim):
        for j in range (omega_dim):
            omega[i,j]=0.0
            for k in range (num_loops+1):
                omega[i,j]+=y[i+1,j+1,k] * loop[k]

    little_omega=np.zeros ((omega_dim),dtype=complex)
    for i in range (omega_dim):
        little_omega[i]=0.0
        for j in range (litte_omega_dim+1):
            for k in range (j,litte_omega_dim+1):
                little_omega[i]+= z[i+1,j,k]*loop[j]*loop[k]
    #little_omega+=- omega[:,2]/2.0-omega[:,4]/2.0 + c_coupling*omega[:,3]-g3_coupling*omega[:,5]/3.0-g3_coupling*omega[:,8]/3.0-g4_coupling*omega[:,9]-g4_coupling*omega[:,14]

    """
    det=np.linalg.det(omega)
    c=np.array(omega)
    #print (c)
    for j in range(omega_dim):
        c[:,j]=little_omega
        LnJ[j]=np.linalg.det(c)/det
        c[:,j]=omega[:,j]
    """

    LnJ=np.linalg.solve(omega,little_omega)
    ##print ("LnJ", LnJ)

    # Obtain value of collective effective potential
    #large_n_energy = np.dot(little_omega , LnJ)/8.0 + loop[3]/2. + loop[5]/2.
    #print ("sarted dervi wrt master variables")
    grad_array_1=np.zeros((num_loops+1,nt_hooft),dtype=complex)##
    grad_array_2=np.zeros((num_loops+1,nt_hooft,nt_hooft),dtype=complex)
    grad_list_2=np.zeros((num_loops+1,nt_hooft*nt_hooft),dtype=complex)

    grad_array_1[1]= np.ones(nt_hooft)    # this assumes first 3 loops are [] [1] [2]
    grad_array_2[2]= np.identity(nt_hooft)
    grad_list_2[2]=(grad_array_2[2]).flatten()
    for i in range (3,num_loops+1):    #assumes first 3 loops will alwyas be the same
    #loop_matrix=matrices_array[loop_list[i][0]-1]
    #print (loop_list[i], loop_list[i][0]-1, len(loop_list[i]))
    #print (loop_matrix)
        for k in range(len(loop_list[i])):
        #print (k)
        #loop_matrix=loop_matrix @ matrices_array[loop_list[i][k]-1]
            loop_left = loop_list[i][0:k]
        #print (loop_left)
            loop_right= loop_list[i][k+1:len(loop_list[i])]
        #print (loop_right)
            loop_grad=np.concatenate ((loop_right , loop_left))
            ##print("loop",i,
                ##"bit matrix", k,
                ##"loop grad", loop_grad)
            grad_len = len(loop_grad)
            grad_matrix=matrices_array[loop_grad[0]-1] # start grad matrix with first matrix.
            ##print("started with matrix",loop_grad[0] )
            if grad_len > 1 :                             # add other matrices to grad matrix.
                for l in range(1,grad_len):
                    if loop_grad[l]==1 :
                        grad_matrix=grad_matrix * x_N_2[0:nt_hooft]          #[k]#matrices_list[loop_list[i][k]-1]
                    else:
                        grad_matrix=np.dot(grad_matrix,matrices_array[1])
            if loop_list[i][k]==1 :                     # if 
                #grad_array[i,0]+=grad_matrix.T
                grad_array_1[i]+=np.diagonal(grad_matrix.real)
                #print("der wrt 1 ", grad_array_1[i])
            else :
                """
                previous code
                grad_array_2[i]+=grad_matrix.T
                print("full der wrt 2", grad_array_2[i])
                real_matrix_2 = np.triu(grad_array_2[i].real)
                #print ("real der wrt 2", real_matrix_2)
                imag_matrix_2 = np.tril(grad_array_2[i].imag)
                #print ("imag der wrt 2", imag_matrix_2)
                #diag_matrix_2 = np.diag(np.diag(grad_array_2[i]))
                #print ("diagonal part", diag_matrix_2)
                #print (real_matrix_2+imag_matrix_2)
                #print (np.shape(real_matrix_2+imag_matrix_2))
                grad_list_2[i]=(real_matrix_2+imag_matrix_2).flatten()#+diag_matrix_2)
                """
                grad_array_2[i]+=grad_matrix

        ##print("der wrt 1 ", grad_array_1[i])
        ##print("full der wrt 2", grad_array_2[i])
        upper_matrix_2 = np.triu((grad_array_2[i] + grad_array_2[i].T))- np.diag(np.diag(grad_array_2[i]))
        lower_matrix_2 = -1.0j*(np.tril((grad_array_2[i] - grad_array_2[i].T)))
        grad_array_2[i] = upper_matrix_2+lower_matrix_2
        ##print("full der wrt 2 after projection ", grad_array_2[i])
        grad_list_2[i]=(grad_array_2[i]).flatten()

        """
    for i in range (num_loops+1):
        print("Loop ", i , "Matrix 1", grad_array_1[i])
    for i in range (num_loops+1):
        print("Loop ", i , "Matrix 2", grad_array_2[i])
        print("Loop ", i , "Matrix 2 flat", grad_list_2[i])
            #grad_matrix_1
            """
            #loop_matrix=np.dot(loop_matrix,matrices_array[loop_list[i][k]-1])
    #print("after k loop, loop matrix", loop_matrix)
        #loop[i]=np.trace(loop_matrix) /( np.power(nt_hooft, (len(loop_list[i])+2)/2)) #note: not declared as complex; always returns real value
    #print ("loop[",i,"]", loop[i])


    #print ("starte grad_1")
    grad_1=np.zeros((nt_hooft),dtype=complex)# dtype=np.float64)

    for i in range (omega_dim):
        for j in range (omega_dim):
            for k in range (num_loops+1):
                #print ("i=",i,"j=",j,"k=", k, "y", y[i+1,j+1,k], y[i+1,j+1,k]* grad_array_1[k])
                grad_1+= - LnJ[i]*y[i+1,j+1,k]*LnJ[j]* grad_array_1[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))

    """
    #print ("first grad_1", grad_1)
    grad_1=np.sum (np.array([[[(- LnJ[i]*y[i+1,j+1,k]*LnJ[j]* grad_array_1[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for j in range (omega_dim)] for k in range (num_loops+1)]),axis=(0,1,2))
    """

    """ old code
    for i in range (2,omega_dim):#(2,omega_size)
        for j in range (litte_omega_dim+1):
            for k in range (litte_omega_dim+1):
                grad_1+=2*LnJ[i]*z[i+1,j,k]*(grad_array_1[j]*loop[k]/(np.power(nt_hooft,(len(loop_list[j])+2)/2))+ loop[j]*grad_array_1[k]/(np.power(nt_hooft, (len(loop_list[k])+2)/2)))
    """

    #print ("starte grad_1 little_omega")

    for i in range (omega_dim):
        for j in range (litte_omega_dim+1):
            for k in range (j,litte_omega_dim+1):
                grad_1+=2*LnJ[i]*z[i+1,j,k]*(grad_array_1[j]*loop[k]/(np.power(nt_hooft,(len(loop_list[j])+2)/2))+ loop[j]*grad_array_1[k]/(np.power(nt_hooft, (len(loop_list[k])+2)/2)))

    #for i in range (omega_dim):
        #for k in range (num_loops+1):
            #grad_1+=(-2*LnJ[i]*(y[i+1,3,k]/2.0 +y[i+1,5,k]/2.0 - c_coupling*y[i+1,4,k]+g3_coupling*y[i+1,6,k]/3.0+g3_coupling*y[i+1,9,k]/3.0+g4_coupling*y[i+1,10,k]+g4_coupling*y[i+1,15,k])*grad_array_1[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2)))
    grad_1=grad_1/8.0+ (1/2)*grad_array_1[3]/ (np.power(nt_hooft, (len(loop_list[3])+2)/2)) + (1/2)*grad_array_1[5]/ (np.power(nt_hooft, (len(loop_list[5])+2)/2)) + g4_coupling*(grad_array_1[10]/ (np.power(nt_hooft, (len(loop_list[10])+2)/2)) )+g4_coupling*(grad_array_1[15]/ (np.power(nt_hooft, (len(loop_list[15])+2)/2)) )


    """
    grad_1+=np.sum (np.array([[[2*LnJ[i]*z[i+1,j,k]*(grad_array_1[j]*loop[k]/(np.power(nt_hooft,(len(loop_list[j])+2)/2))+ loop[j]*grad_array_1[k]/(np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for j in range (litte_omega_dim+1)] for k in range (litte_omega_dim+1)]),axis=(0,1,2))
    grad_1+=np.sum (np.array([[(-2*LnJ[i]*(y[i+1,3,k]/2.0 +y[i+1,5,k]/2.0 - c_coupling*y[i+1,4,k]+g3_coupling*y[i+1,6,k]/3.0+g3_coupling*y[i+1,9,k]/3.0+g4_coupling*y[i+1,10,k]+g4_coupling*y[i+1,15,k])*grad_array_1[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for k in range (num_loops+1)]),axis=(0,1))
    grad_1=grad_1/8.0
    """
    grad_2=np.zeros((nt_hooft*nt_hooft),dtype=complex)

    #print ("started grad_2")
    #""" older version - slower but less memory
    for i in range (omega_dim):
        for j in range (omega_dim):
            for k in range (num_loops+1):
                grad_2+= - LnJ[i]*y[i+1,j+1,k]*LnJ[j]* grad_list_2[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))
    ##print ("grad_2 ", grad_2)
    #print ("finished first grad_2")
    """

    grad_2=np.sum (np.array([[[(- LnJ[i]*y[i+1,j+1,k]*LnJ[j]* grad_list_2[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for j in range (omega_dim)] for k in range (num_loops+1)]),axis=(0,1,2))
    """
    """ old code
    for i in range (2,omega_dim):#(2,omega_size)
        for j in range (litte_omega_dim+1):
            for k in range (litte_omega_dim+1):
                grad_2+=2*LnJ[i]*z[i+1,j,k]*(grad_list_2[j]*loop[k]/(np.power(nt_hooft, (len(loop_list[j])+2)/2))+ loop[j]*grad_list_2[k]/(np.power(nt_hooft, (len(loop_list[k])+2)/2)))

    ##print ("second grad_2 ", grad_2)
    """
    for i in range (omega_dim):
        for j in range (litte_omega_dim+1):
            for k in range (j,litte_omega_dim+1):
                grad_2+=2*LnJ[i]*z[i+1,j,k]*(grad_list_2[j]*loop[k]/(np.power(nt_hooft,(len(loop_list[j])+2)/2))+ loop[j]*grad_list_2[k]/(np.power(nt_hooft, (len(loop_list[k])+2)/2)))

    #for i in range (omega_dim):
        #for k in range (num_loops+1):
            #grad_2+=(- 2*LnJ[i]*(y[i+1,3,k]/2.0 +y[i+1,5,k]/2.0 - c_coupling*y[i+1,4,k]+g3_coupling*y[i+1,6,k]/3.0+g3_coupling*y[i+1,9,k]/3.0+g4_coupling*y[i+1,10,k]+g4_coupling*y[i+1,15,k])*grad_list_2[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2)))
    grad_2=grad_2/8.0+ (1/2)*grad_list_2[3]/ (np.power(nt_hooft, (len(loop_list[3])+2)/2)) + (1/2)*grad_list_2[5]/ (np.power(nt_hooft, (len(loop_list[5])+2)/2)) + g4_coupling*grad_list_2[10]/ (np.power(nt_hooft, (len(loop_list[10])+2)/2))+ g4_coupling*grad_list_2[15]/ (np.power(nt_hooft, (len(loop_list[15])+2)/2))


    """
    #print ("starte grad_1 little_omega")
    grad_2+=np.sum (np.array([[[2*LnJ[i]*z[i+1,j,k]*(grad_list_2[j]*loop[k]/(np.power(nt_hooft,(len(loop_list[j])+2)/2))+ loop[j]*grad_list_2[k]/(np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for j in range (litte_omega_dim+1)] for k in range (litte_omega_dim+1)]),axis=(0,1,2))
    #grad_2+=np.sum (np.array([[(- 2*LnJ[i]*(y[i+1,3,k]/2.0 +y[i+1,5,k]/2.0 - c_coupling*y[i+1,4,k]+g3_coupling*y[i+1,6,k]/3.0+g3_coupling*y[i+1,9,k]/3.0)*grad_list_2[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for k in range (num_loops+1)]),axis=(0,1))
    grad_2+=np.sum (np.array([[(- 2*LnJ[i]*(y[i+1,3,k]/2.0 +y[i+1,5,k]/2.0 - c_coupling*y[i+1,4,k]+g3_coupling*y[i+1,6,k]/3.0+g3_coupling*y[i+1,9,k]/3.0+g4_coupling*y[i+1,10,k]+g4_coupling*y[i+1,15,k])*grad_list_2[k]/ (np.power(nt_hooft, (len(loop_list[k])+2)/2))) for i in range (omega_dim)] for k in range (num_loops+1)]),axis=(0,1))
    grad_2=grad_2/8.0
    #grad_2=grad_2/8.0 + grad_list_2[3]/ (2* np.power(nt_hooft, (len(loop_list[3])+2)/2))  + grad_list_2[5]/ (2* np.power(nt_hooft, (len(loop_list[5])+2)/2))
    #grad_2=grad_2.flatten()
    ##print ("third grad_2 ", grad_2)
    #print ("Concatenated derivative", format(np.concatenate((grad_1.real,grad_2.real),axis=0),'.6f'))
    """
    """
    print ("%%%%%%% Concatenated gradient")
    for x in np.concatenate((grad_1.real,grad_2.real),axis=0):
        print (format(x,'.8f'))
    """
    #""" most recent
    #print ("%%%%%%% Concatenated gradient")
    #for x in np.concatenate((grad_1,grad_2),axis=0):
        #print (x)
    #"""

    #print ("Concatenated derivative", np.around(np.concatenate((grad_1.real,grad_2.real),axis=0),decimals=6))
    return np.concatenate((grad_1.real,grad_2.real),axis=0)
    #x_init=np.concatenate ((eigen_init, m_2_init.flatten()),axis=0)


#working towards minimise effect_pot

> Indented block

> Indented block

> Indented block







In [9]:
start1=timer()
for i in range (1,max_length+1):
    start=end+1
    loops_fixed_length = loops_of_length (i)
    num_fixed_length=np.ma.size(loops_fixed_length, axis=0)
    for j in range(num_fixed_length):
        loop_list.append(loops_fixed_length [j:j+1 ,:])
    end=start+num_fixed_length-1
    numb_tuples_list.append((start,end))
end1=timer()

starting loops of length  1
starting loops of length  2
starting with 1 [2]s
starting loops of length  3
starting with 1 [2]s
starting with 2 [2]s
starting loops of length  4
starting with 1 [2]s
starting with 2 [2]s
starting with 3 [2]s
starting loops of length  5
starting with 1 [2]s
starting with 2 [2]s
starting with 3 [2]s
starting with 4 [2]s
starting loops of length  6
starting with 1 [2]s
starting with 2 [2]s
starting with 3 [2]s
starting with 4 [2]s
starting with 5 [2]s
starting loops of length  7
starting with 1 [2]s
starting with 2 [2]s
starting with 3 [2]s
starting with 4 [2]s
starting with 5 [2]s
starting with 6 [2]s
starting loops of length  8
starting with 1 [2]s
starting with 2 [2]s
starting with 3 [2]s
starting with 4 [2]s
starting with 5 [2]s
starting with 6 [2]s
starting with 7 [2]s
starting loops of length  9
starting with 1 [2]s
starting with 2 [2]s
starting with 3 [2]s
starting with 4 [2]s
starting with 5 [2]s
starting with 6 [2]s
starting with 7 [2]s
starting with

In [10]:
loops_fixed_length

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 2],
       [1, 1, 1, ..., 1, 2, 2],
       ...,
       [1, 2, 2, ..., 2, 2, 2],
       [1, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2]])

In [11]:
loop_list

[array([], dtype=float64),
 array([[1]]),
 array([[2]]),
 array([[1, 1]]),
 array([[1, 2]]),
 array([[2, 2]]),
 array([[1, 1, 1]]),
 array([[1, 1, 2]]),
 array([[1, 2, 2]]),
 array([[2, 2, 2]]),
 array([[1, 1, 1, 1]]),
 array([[1, 1, 1, 2]]),
 array([[1, 1, 2, 2]]),
 array([[1, 2, 1, 2]]),
 array([[1, 2, 2, 2]]),
 array([[2, 2, 2, 2]]),
 array([[1, 1, 1, 1, 1]]),
 array([[1, 1, 1, 1, 2]]),
 array([[1, 1, 1, 2, 2]]),
 array([[1, 1, 2, 1, 2]]),
 array([[1, 1, 2, 2, 2]]),
 array([[1, 2, 1, 2, 2]]),
 array([[1, 2, 2, 2, 2]]),
 array([[2, 2, 2, 2, 2]]),
 array([[1, 1, 1, 1, 1, 1]]),
 array([[1, 1, 1, 1, 1, 2]]),
 array([[1, 1, 1, 1, 2, 2]]),
 array([[1, 1, 1, 2, 1, 2]]),
 array([[1, 1, 2, 1, 1, 2]]),
 array([[1, 1, 1, 2, 2, 2]]),
 array([[1, 1, 2, 1, 2, 2]]),
 array([[1, 1, 2, 2, 1, 2]]),
 array([[1, 2, 1, 2, 1, 2]]),
 array([[1, 1, 2, 2, 2, 2]]),
 array([[1, 2, 1, 2, 2, 2]]),
 array([[1, 2, 2, 1, 2, 2]]),
 array([[1, 2, 2, 2, 2, 2]]),
 array([[2, 2, 2, 2, 2, 2]]),
 array([[1, 1, 1, 1, 1, 1

In [12]:
print(numb_tuples_list)

[(0, 0), (1, 2), (3, 5), (6, 9), (10, 15), (16, 23), (24, 37), (38, 57), (58, 93), (94, 153), (154, 261), (262, 449), (450, 801), (802, 1433), (1434, 2615)]


In [13]:
#  Change (1,n) shape to (n,) for easier loop manipulation
for i in range(len(loop_list)):
    temp_loop = loop_list[i].T
    temp_len = np.ma.size(temp_loop, axis=0)
    loop_list[i] = np.reshape(temp_loop,(temp_len,))

In [14]:
i=0
for x in loop_list:
    print ("Loop", i,"=", x)
    i+=1

Loop 0 = []
Loop 1 = [1]
Loop 2 = [2]
Loop 3 = [1 1]
Loop 4 = [1 2]
Loop 5 = [2 2]
Loop 6 = [1 1 1]
Loop 7 = [1 1 2]
Loop 8 = [1 2 2]
Loop 9 = [2 2 2]
Loop 10 = [1 1 1 1]
Loop 11 = [1 1 1 2]
Loop 12 = [1 1 2 2]
Loop 13 = [1 2 1 2]
Loop 14 = [1 2 2 2]
Loop 15 = [2 2 2 2]
Loop 16 = [1 1 1 1 1]
Loop 17 = [1 1 1 1 2]
Loop 18 = [1 1 1 2 2]
Loop 19 = [1 1 2 1 2]
Loop 20 = [1 1 2 2 2]
Loop 21 = [1 2 1 2 2]
Loop 22 = [1 2 2 2 2]
Loop 23 = [2 2 2 2 2]
Loop 24 = [1 1 1 1 1 1]
Loop 25 = [1 1 1 1 1 2]
Loop 26 = [1 1 1 1 2 2]
Loop 27 = [1 1 1 2 1 2]
Loop 28 = [1 1 2 1 1 2]
Loop 29 = [1 1 1 2 2 2]
Loop 30 = [1 1 2 1 2 2]
Loop 31 = [1 1 2 2 1 2]
Loop 32 = [1 2 1 2 1 2]
Loop 33 = [1 1 2 2 2 2]
Loop 34 = [1 2 1 2 2 2]
Loop 35 = [1 2 2 1 2 2]
Loop 36 = [1 2 2 2 2 2]
Loop 37 = [2 2 2 2 2 2]
Loop 38 = [1 1 1 1 1 1 1]
Loop 39 = [1 1 1 1 1 1 2]
Loop 40 = [1 1 1 1 1 2 2]
Loop 41 = [1 1 1 1 2 1 2]
Loop 42 = [1 1 1 2 1 1 2]
Loop 43 = [1 1 1 1 2 2 2]
Loop 44 = [1 1 1 2 1 2 2]
Loop 45 = [1 1 1 2 2 1 2]
Loop 46 =

In [15]:
#  Building Omega matrix
omega_size = numb_tuples_list [omega_length][1]
print ("Omega size = ", omega_size)
max_size = numb_tuples_list [max_length][1]
print ("Total number of loops = ", max_size)
y=np.zeros((omega_size+1, omega_size+1, max_size+1))

Omega size =  93
Total number of loops =  2615


In [ ]:
print ("Joining loops and generating Omega ")

for i in range (1,omega_size+1):
    print ("starting row = ", i)
    for j in range (i,omega_size+1):
        loop_1 = loop_list [i]
        loop_2 = loop_list [j]
        #print ("loop_1", loop_1, "loop_2", loop_2    )
        for k in range (len(loop_1)):
            for l in range (len(loop_2)):
                if loop_1[k]==loop_2[l]:
                    loop_1_left = loop_1[0:k]
                    #print (loop_1_left)
                    loop_1_right= loop_1[k+1:len(loop_1)]
                    #print (loop_1_right)
                    loop_2_left = loop_2[0:l]
                    #print (loop_2_left)
                    loop_2_right= loop_2[l+1:len(loop_2)]
                    #print (loop_2_right)
                    #joined_loop = np.concatenate ((loop_1_right , loop_2_left,loop_2_right ,loop_1_left ))
                    joined_loop = np.concatenate ((loop_1_right , loop_1_left,loop_2_right ,loop_2_left ))
                    #print (str(loop_1), str(loop_2), (joined_loop))
                    start=numb_tuples_list [len(joined_loop)][0]
                    #print(start)
                    end=numb_tuples_list [len(joined_loop)][1]
                    #print (end)
                    for p in range(start,end+1):
                        if compare (loop_list[p],joined_loop):
                            #print ("identified loop")
                            y[i][j][p]+=1
                            if i != j : y[j][i][p]+=1

                            #print (y[i][j][p])

for i in range (1,omega_size+1):
    for j in range (1,omega_size+1):
        for k in range (max_size+1):
            if (y[i][j][k])!= 0. :
                print ("Loop", i,"=",loop_list[i],"joins with", "Loop", j,"=", loop_list[j], "into Loop", k,"=", loop_list[k], (y[i][j][k]), "times." )



Streaming output truncated to the last 5000 lines.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1361 = [1 1 2 1 2 2 2 1 2 2 2 2 2] 1.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1362 = [1 1 2 1 2 2 2 2 1 2 2 2 2] 1.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1365 = [1 1 2 1 2 2 2 2 2 2 2 1 2] 2.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1379 = [1 1 2 2 2 2 1 1 2 2 2 2 2] 1.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1383 = [1 1 2 2 2 2 1 2 2 2 2 1 2] 1.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1386 = [1 1 2 2 2 2 2 1 2 2 2 1 2] 1.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1390 = [1 2 1 2 1 2 1 2 2 2 2 2 2] 4.0 times.
Loop 54 = [1 2 1 2 2 2 2] joins with Loop 89 = [1 2 1 2 2 2 2 2] into Loop 1391 = [1 2 1 2 1 2 2 1 2 2

In [ ]:
loop_list

In [ ]:
print ("Little omega")
#size to be fixed
little_omega_size = numb_tuples_list [omega_length-2][1]
print ("little_omega_size = ", little_omega_size)
#z=np.zeros ((omega_size+1,little_omega_size+1, little_omega_size+1))
z=np.zeros ((omega_size+1,omega_size+1, omega_size+1))

for i in range (3,omega_size+1):
    for k in range (len(loop_list[i])):
        for l in range (len(loop_list[i])):
            if k!=l :
                if loop_list[i][k]== loop_list[i][l]:
                    if k<l :
                        loop_inside=loop_list[i][k+1:l]
                        #print (i,loop_list[i], k,l, loop_inside)
                        #loop_outside_1 =
                        loop_outside=np.concatenate((loop_list[i][l+1:len(loop_list[i])],loop_list[i][0:k]))
                        #print (i,loop_list[i], k,l, loop_inside, loop_outside)
                        start=numb_tuples_list [len(loop_inside)][0]
                        end=numb_tuples_list [len(loop_inside)][1]
                        for p in range(start,end+1):
                            if compare (loop_list[p],loop_inside): loop_inside_index=p
                        start=numb_tuples_list [len(loop_outside)][0]
                        end=numb_tuples_list [len(loop_outside)][1]
                        for p in range(start,end+1):
                            if compare (loop_list[p],loop_outside):loop_outside_index=p
                        #z[i][loop_inside_index][loop_outside_index]+=1
                    else:
                        loop_inside=loop_inside=loop_list[i][l+1:k]
                        loop_outside=np.concatenate((loop_list[i][k+1:len(loop_list[i])],loop_list[i][0:l] ) )
                        #print (i,loop_list[i],k,l, loop_inside, loop_outside)
                        start=numb_tuples_list [len(loop_inside)][0]
                        end=numb_tuples_list [len(loop_inside)][1]
                        for p in range(start,end+1):
                            if compare (loop_list[p],loop_inside): loop_inside_index=p
                        start=numb_tuples_list [len(loop_outside)][0]
                        end=numb_tuples_list [len(loop_outside)][1]
                        for p in range(start,end+1):
                            if compare (loop_list[p],loop_outside):loop_outside_index=p
                    z[i][loop_inside_index][loop_outside_index]+=1


"""
for i in range (1,omega_size+1):
    for j in range (little_omega_size+1):
        for k in range (little_omega_size+1):
            if (z[i][j][k])!= 0. :
                print ("Loop", i,"=",loop_list[i],"breaks into", "Loop", j,"=", loop_list[j], "and Loop", k,"=", loop_list[k], (z[i][j][k]), "times." )
"""
# turn z into upper diagonal
for i in range (1,omega_size+1):
    for j in range (little_omega_size+1):
        for k in range (j+1,little_omega_size+1):
            z[i,j,k] = z[i,j,k]+ z[i,k,j]
            z[i,k,j] = 0


print("Symmetrized little omega")
for i in range (1,omega_size+1):
    for j in range (little_omega_size+1):
        for k in range (little_omega_size+1):
            if (z[i][j][k])!= 0. :
                print ("Loop", i,"=",loop_list[i],"breaks into", "Loop", j,"=", loop_list[j], "and Loop", k,"=", loop_list[k], (z[i][j][k]), "times." )


In [ ]:
loop=np.zeros ((max_size+1),dtype=complex)
omega=np.zeros ((omega_size,omega_size),dtype=complex)
#omega=np.diag(np.ones((omega_size))*(1.e-13))
#print ("Omega initial", omega )
little_omega=np.zeros ((omega_size),dtype=complex)
LnJ=np.zeros ((omega_size),dtype=complex)

In [ ]:
print("Main Program")

In [ ]:
eigen_init=(np.random.rand(N) - np.ones(N)*0.5)*np.sqrt(N)
m_2_init=(np.random.rand(N,N)-np.ones((N,N))*0.5)*np.sqrt(N)
x_init=np.concatenate ((eigen_init, m_2_init.flatten()),axis=0)
x_init

In [ ]:
x_init.shape

In [ ]:
start = timer()
#Energy = minimize (effect_pot, x_init, args=(omega_size,max_size ,little_omega_size, N, 0.0), method='nelder-mead', options={'disp': True, 'maxfev': 50000, 'maxiter': 50000})
Energy = minimize (effect_pot, x_init, args=(omega_size,max_size, little_omega_size, N, c, g_3,g_4), method='BFGS', jac=effect_pot_grad, options={'disp': True, 'maxiter': 5000,'gtol': np.sqrt(N*(N+1))*1e-8})
#Energy = minimize (effect_pot, x_init, args=(omega_size,max_size, little_omega_size, N, c, g_3,g_4), method='L-BFGS-B', jac=effect_pot_grad, options={'disp': True, 'maxiter': 5000,'gtol': np.sqrt(N*(N+1))*1e-16})
#Energy = minimize (effect_pot, x_init, args=(omega_size,max_size, little_omega_size, N, c, g_3,g_4), method='CG', jac=effect_pot_grad, options={'disp': True, 'maxiter': 5000,'gtol': np.sqrt(N*(N+1))*1e-05})
#Energy = minimize (EffectPot, EigenInit, args=(OmegaSizeInit, N, g), method='BFGS', jac=Grad, options={'disp' : True, 'maxiter': 5000})

end = timer()
print((end - start), "sec in minimization")

In [ ]:
matrix_1_final = np.diag(Energy.x[0:N])

##print ("Matrix 1")
##print (np.around(matrix_1_final,decimals=5))


temp_matrix_2_final = np.reshape (Energy.x[N:N**2+N], (N,N))
real_matrix_2_final = np.triu(temp_matrix_2_final)
real_matrix_2_final = real_matrix_2_final+real_matrix_2_final.T-np.diag(np.diag(temp_matrix_2_final))
imag_matrix_2_final = np.tril(temp_matrix_2_final)
imag_matrix_2_final = imag_matrix_2_final-imag_matrix_2_final.T
matrix_2_final=real_matrix_2_final+imag_matrix_2_final*1.j

##print ("Matrix 2")
##print (np.around(matrix_2_final,decimals=5))

matrices_array_final =np.zeros((2,N,N),dtype=complex)
matrices_array_final[0] =matrix_1_final
matrices_array_final[1] =matrix_2_final

for i in range (1,max_size+1):
    loop_matrix=matrices_array_final[loop_list[i][0]-1]
    #print (loop_list[i], loop_list[i][0]-1, len(loop_list[i]))
    #print (loop_matrix)
    for k in range(1,len(loop_list[i])):
        #print (k)
        #loop_matrix=loop_matrix @ matrices_array[loop_list[i][k]-1]
        loop_matrix=np.dot(loop_matrix,matrices_array_final[loop_list[i][k]-1])
    #print("after k loop, loop matrix", loop_matrix)
    loop[i]=np.trace(loop_matrix) / ( np.power(N, (len(loop_list[i])+2)/2)) #note: not declared as complex; always returns real value
    #print ("loop[",i,"]", loop[i])
loop[0]=1.

print ("Energy =", Energy.fun)
for i in range (max_size+1):
    print ("Loop,", i,",=,",loop_list[i], "=", format(loop[i].real, '.6f'), "", format(loop[i].imag, '.6f')+"j")#round (loop[i].real,5)


In [ ]:
i=0
reals=[]
imags=[]
for i in range(max_size+1):
    print (format(loop[i].real, '.6f'))
    reals.append('%.6f' % float(loop[i].real))
    imags.append('%.6f' % float(loop[i].imag))
    i+=1

In [ ]:
reals

In [ ]:
imags